In [44]:
from bs4 import BeautifulSoup as bs
import requests
import json
from datetime import datetime
import typer
import math
import re
import sys
import csv
data_csv = {'titulo':[],'empresa':[],'descrição':[],'data de publicação':[],'salário':[],'localização':[]}
import os    
import time

API_KEY = "9fa7ce317d6e85c90d92244adb9146c6"
BASE_URL = "https://api.itjobs.pt/job/list.json"

app = typer.Typer()

def fetch_jobs(params):
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(BASE_URL, params=params, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao obter dados: {response.status_code} - {response.text}")
        return None
    
def top10_data():
  URL = 'https://api.itjobs.pt/job/list.json?api_key=9fa7ce317d6e85c90d92244adb9146c6'
  request = requests.get(URL,headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
  data = request.json()
  alldata={}
  total_pages = math.ceil(data['total']/12)
  for rep in range(total_pages):
      datasets = requests.get(URL, params={'limit': 12, 'page':rep+1},headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'}).json()
      results = datasets['results']
      if 'csv' in sys.argv:        
        for item in results:
          alldata[item['id']] = [item.get('title', ''),item['company'].get('name', ''), item['company'].get('description', ''),item.get('publishedAt', ''),item.get('wage', ''),item['company'].get('address', '')]
      else: 
        for item in results:
          alldata[item['id']] = [item['company']['name'],item['publishedAt']]
  return alldata


def search_data(company, location, num_jobs):
    print(f"Procurando trabalhos para Empresa: {company}, Localidade: {location}, Número de Trabalhos: {num_jobs}")
    
    # URL base da API
    URL = 'https://api.itjobs.pt/job/list.json'
    API_KEY = '9fa7ce317d6e85c90d92244adb9146c6'  # Substitua pela sua chave de API

    # Inicializa a lista de trabalhos encontrados
    found_jobs = []
    
    # Faz a primeira requisição para saber o total de trabalhos
    response = requests.get(URL, params={'api_key': API_KEY, 'limit': 12, 'page': 1}, headers={'User-Agent': 'Mozilla/5.0'})
    data = response.json()
    
    if 'total' in data and data['total'] > 0:
        total_pages = math.ceil(data['total'] / 12)
        for rep in range(total_pages):
            # Requisição para cada página
            datasets = requests.get(URL, params={'api_key': API_KEY, 'limit': 12, 'page': rep + 1}, headers={'User-Agent': 'Mozilla/5.0'}).json()
            results = datasets.get('results', [])
            
            for item in results:
                # Acessa o nome da empresa e localidade
                company_name = item['company']['name']
                job_locations = [loc['name'] for loc in item.get('locations', [])]
                
                # Filtra pelos critérios
                if (company_name == company) and (location in job_locations):
                    found_jobs.append(item)
                    print(f"Trabalho encontrado: {item['title']} na empresa {company_name} na localidade {location}")
                
                # Limita a quantidade de trabalhos encontrados
                if len(found_jobs) >= num_jobs:
                    break
            
            # Para o loop se já encontrou o número de trabalhos desejado
            if len(found_jobs) >= num_jobs:
                break

    # Verifica se encontrou trabalhos
    if found_jobs:
        return found_jobs
    else:
        print("Nenhum trabalho encontrado para os critérios especificados.")
        return None












def salary(jobID):
    URL = 'https://api.itjobs.pt/job/get.json'
    api_key = '9fa7ce317d6e85c90d92244adb9146c6'
    
    try:
        # Realiza a requisição para obter informações sobre o trabalho
        response = requests.get(URL, params={'api_key': api_key, 'id': jobID}, 
                                headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
        
        # Verifica o código de status da resposta
        if response.status_code != 200:
            print(f"Erro: Status HTTP {response.status_code}. Verifique a chave da API e o jobID.")
            return
        
        # Tenta carregar o conteúdo JSON da resposta
        data = response.json()
        
    except requests.JSONDecodeError:
        print("Erro: A resposta não está em formato JSON. Verifique a URL e os parâmetros.")
        return

    # Verifica se a resposta contém o campo 'error' indicando que o jobID não foi encontrado
    if 'error' in data:
        print(f"Erro: {data['error'].get('message', 'Erro desconhecido')}")
        return

    # Tenta extrair o salário do campo 'wage' se presente
    wage = data.get('wage')
    if wage is not None:
        print(f"Apresenta um salário inicial de {wage}€.")
        return
    
    # Caso 'wage' esteja ausente, tenta encontrar o salário em campos alternativos usando expressões regulares
    salary_found = False
    fields_to_check = ['description', 'salary_info', 'benefits']
    
    for field in fields_to_check:
        if field in data and isinstance(data[field], str):  # Verifica se o campo é uma string
            match = re.search(r'(\d+(\.\d{1,2})?)\s*(€|USD|BRL|GBP)', data[field])
            if match:
                print(f"Salário encontrado no campo {field}: {match.group(0)}")
                salary_found = True
                break
    
    # Se não encontrou o salário em nenhum dos campos alternativos
    if not salary_found:
        # Tenta fornecer informações adicionais sobre o contrato
        contract_info = []
        if 'contracts' in data and isinstance(data['contracts'], list) and data['contracts']:
            contract_info.append(f"Contrato: {data['contracts'][0].get('name', 'Desconhecido')}")
        if 'types' in data and isinstance(data['types'], list) and data['types']:
            contract_info.append(f"Tipo: {data['types'][0].get('name', 'Desconhecido')}")
        
        # Se houver informações de contrato ou tipo, exibe-as; caso contrário, informa que não há dados disponíveis
        if contract_info:
            print("Sem informação sobre o salário.\n" + ", ".join(contract_info))
        else:
            print("Sem informação sobre o trabalho selecionado.")

def filter_jobs_by_skills_and_date(skills, start_date, end_date):
    URL = 'https://api.itjobs.pt/job/list.json'
    API_KEY = '9fa7ce317d6e85c90d92244adb9146c6'  # Substitua pela sua chave de API
    
    # Inicializa uma lista para armazenar trabalhos encontrados
    found_jobs = []

    # Faz a primeira requisição para saber o total de trabalhos
    response = requests.get(URL, params={'api_key': API_KEY, 'limit': 12, 'page': 1}, headers={'User-Agent': 'Mozilla/5.0'})
    data = response.json()
    
    # Verifica o número total de páginas
    if 'total' in data:
        total_pages = (data['total'] // 12) + 1
        for page in range(1, total_pages + 1):
            response = requests.get(URL, params={'api_key': API_KEY, 'limit': 12, 'page': page}, headers={'User-Agent': 'Mozilla/5.0'})
            jobs_data = response.json().get('results', [])
            
            for job in jobs_data:
                job_date = datetime.strptime(job['publishedAt'], '%Y-%m-%d %H:%M:%S')
                job_skills = job.get('body', '')

                # Verifica se a data está no intervalo desejado
                if start_date <= job_date <= end_date:
                    # Verifica se todas as habilidades estão no campo `body`
                    if all(re.search(skill, job_skills, re.IGNORECASE) for skill in skills):
                        found_jobs.append({
                            "title": job.get('title', ''),
                            "company": job['company'].get('name', ''),
                            "description": job.get('body', ''),
                            "publishedAt": job.get('publishedAt', ''),
                            "wage": job.get('wage', ''),
                            "location": job['company'].get('address', '')
                        })
    
    # Retorna a lista de trabalhos encontrados
    return found_jobs



















def skills_data():
  not_found = 1
  URL = 'https://api.itjobs.pt/job/list.json?api_key=2fd9dd6db7e14adbf04df55811af5d22'
  request = requests.get(URL,headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
  data = request.json()
  alldata={}
  total_pages = math.ceil(data['total']/12)
  for rep in range(total_pages):
      datasets = requests.get(URL, params={'limit': 12, 'page':rep+1},headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'}).json()
      results = datasets['results']
      for item in results:
            data_trabalho = dt.strptime(item['publishedAt'],'%Y-%m-%d %H:%M:%S')
            if data_trabalho > data_ini and data_trabalho < data_fim:
                for query in queries:
                    palavra_chave = re.search(query,item['body'])
                    if palavra_chave:
                      if csv_check == 0:
                        alldata[item['id']] = [item['company']['name'],item['company']['url']]
                        not_found = 0
                      else:
                        alldata[item['id']] = [item.get('title', ''),item['company'].get('name', ''), item['company'].get('description', ''),item.get('publishedAt', ''),item.get('wage', ''),item['company'].get('address', '')]
                        not_found = 0
            elif data_trabalho < data_ini:
                not_found = 0
                return alldata
  if not_found == 1:
      return False
  return alldata
  













######################################################################"CSV"#################################################################################################

def top10_data(export_csv=False):
    URL = 'https://api.itjobs.pt/job/list.json?api_key=9fa7ce317d6e85c90d92244adb9146c6'
    
    # Faz a requisição inicial para obter o total de páginas
    request = requests.get(URL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
    data = request.json()
    alldata = {}
    total_pages = math.ceil(data['total'] / 12)
    
    # Itera sobre todas as páginas de resultados
    for rep in range(total_pages):
        datasets = requests.get(
            URL, 
            params={'limit': 12, 'page': rep + 1},
            headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'}
        ).json()
        
        results = datasets.get('results', [])
        
        # Adiciona os dados conforme a escolha por exportação CSV
        for item in results:
            if export_csv:
                alldata[item['id']] = [
                    item.get('title', ''),
                    item['company'].get('name', ''),
                    item['company'].get('description', ''),
                    item.get('publishedAt', ''),
                    item.get('wage', ''),
                    item['company'].get('address', '')
                ]
            else:
                alldata[item['id']] = [
                    item['company'].get('name', ''),
                    item.get('publishedAt', '')
                ]

    # Exporta para CSV se a opção estiver ativada
    if export_csv:
        with open('top10_jobs.csv', mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Title', 'Company', 'Description', 'Published At', 'Salary', 'Location'])
            for job_data in alldata.values():
                writer.writerow(job_data)
        print("Dados exportados para top10_jobs.csv.")
    else:
        print("Exportação para CSV não foi solicitada.")

    return alldata
  
def search_data(company, location, num_jobs):
    print(f"Procurando trabalhos para Empresa: {company}, Localidade: {location}, Número de Trabalhos: {num_jobs}")
    
    # URL base da API
    URL = 'https://api.itjobs.pt/job/list.json'
    API_KEY = '9fa7ce317d6e85c90d92244adb9146c6'  # Substitua pela sua chave de API

    # Inicializa a lista de trabalhos encontrados
    found_jobs = []
    
    # Faz a primeira requisição para saber o total de trabalhos
    response = requests.get(URL, params={'api_key': API_KEY, 'limit': 12, 'page': 1}, headers={'User-Agent': 'Mozilla/5.0'})
    data = response.json()
    
    if 'total' in data and data['total'] > 0:
        total_pages = math.ceil(data['total'] / 12)
        for rep in range(total_pages):
            # Requisição para cada página
            datasets = requests.get(URL, params={'api_key': API_KEY, 'limit': 12, 'page': rep + 1}, headers={'User-Agent': 'Mozilla/5.0'}).json()
            results = datasets.get('results', [])
            
            for item in results:
                # Acessa o nome da empresa e localidade
                company_name = item['company']['name']
                job_locations = [loc['name'] for loc in item.get('locations', [])]
                
                # Filtra pelos critérios
                if (company_name == company) and (location in job_locations):
                    found_jobs.append(item)
                    print(f"Trabalho encontrado: {item['title']} na empresa {company_name} na localidade {location}")
                
                # Limita a quantidade de trabalhos encontrados
                if len(found_jobs) >= num_jobs:
                    break
            
            # Para o loop se já encontrou o número de trabalhos desejado
            if len(found_jobs) >= num_jobs:
                break

    # Verifica se encontrou trabalhos
    if found_jobs:
        # Verifica se o argumento 'csv' está presente para exportar os dados para um arquivo CSV
        if 'csv' in sys.argv:
            # Exportar para CSV
            with open('found_jobs.csv', mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(['Title', 'Company', 'Description', 'Published At', 'Salary', 'Location'])
                for item in found_jobs:
                    title = item.get('title', '')
                    company_name = item['company'].get('name', '')
                    description = item['company'].get('description', '')
                    published_at = item.get('publishedAt', '')
                    wage = item.get('wage', '')
                    locations = ', '.join([loc['name'] for loc in item.get('locations', [])])
                    
                    writer.writerow([title, company_name, description, published_at, wage, locations])
            print(f"{len(found_jobs)} trabalhos exportados para 'found_jobs.csv'.")
        return found_jobs
    else:
        print("Nenhum trabalho encontrado para os critérios especificados.")
        return None
    

def salary(jobID):
    URL = 'https://api.itjobs.pt/job/get.json'
    api_key = '9fa7ce317d6e85c90d92244adb9146c6'
    
    try:
        # Realiza a requisição para obter informações sobre o trabalho
        response = requests.get(URL, params={'api_key': api_key, 'id': jobID}, 
                                headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
        
        # Verifica o código de status da resposta
        if response.status_code != 200:
            print(f"Erro: Status HTTP {response.status_code}. Verifique a chave da API e o jobID.")
            return
        
        # Tenta carregar o conteúdo JSON da resposta
        data = response.json()
        
    except requests.JSONDecodeError:
        print("Erro: A resposta não está em formato JSON. Verifique a URL e os parâmetros.")
        return

    # Verifica se a resposta contém o campo 'error' indicando que o jobID não foi encontrado
    if 'error' in data:
        print(f"Erro: {data['error'].get('message', 'Erro desconhecido')}")
        return

    # Tenta extrair o salário do campo 'wage' se presente
    wage = data.get('wage')
    salary_found = False

    # Se o salário for encontrado, imprime e grava no CSV
    if wage is not None:
        print(f"Apresenta um salário inicial de {wage}€.")
        salary_found = True
    else:
        # Caso 'wage' esteja ausente, tenta encontrar o salário em campos alternativos usando expressões regulares
        fields_to_check = ['description', 'salary_info', 'benefits']
        
        for field in fields_to_check:
            if field in data and isinstance(data[field], str):  # Verifica se o campo é uma string
                match = re.search(r'(\d+(\.\d{1,2})?)\s*(€|USD|BRL|GBP)', data[field])
                if match:
                    print(f"Salário encontrado no campo {field}: {match.group(0)}")
                    salary_found = True
                    break

    # Se o salário não for encontrado em nenhum campo
    if not salary_found:
        print("Não foi possível encontrar o salário para este trabalho.")
    
    # Coleta as informações principais do trabalho
    title = data.get('title', '')
    company_name = data['company'].get('name', '')
    description = data['company'].get('description', '')
    published_at = data.get('publishedAt', '')
    locations = ', '.join([loc['name'] for loc in data.get('locations', [])])
    
    # Se a opção 'csv' for passada nos argumentos, exporta para CSV
    if 'csv' in sys.argv:
        with open(f'{jobID}_job_salary.csv', mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Title', 'Company', 'Description', 'Published At', 'Salary', 'Location'])
            
            # Escreve os dados no CSV
            writer.writerow([title, company_name, description, published_at, wage if wage else 'Não disponível', locations])
        
        print(f"Informações exportadas para o arquivo {jobID}_job_salary.csv.")

def skills_data(queries, data_ini, data_fim, csv_check=0):
    not_found = 1
    URL = 'https://api.itjobs.pt/job/list.json?api_key=2fd9dd6db7e14adbf04df55811af5d22'
    
    # Realiza a requisição para obter a lista de trabalhos
    request = requests.get(URL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
    data = request.json()

    alldata = {}
    total_pages = math.ceil(data['total'] / 12)
    
    # Itera pelas páginas dos trabalhos
    for rep in range(total_pages):
        datasets = requests.get(URL, params={'limit': 12, 'page': rep + 1}, 
                                headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'}).json()
        results = datasets['results']
        
        # Verifica os trabalhos para cada item
        for item in results:
            data_trabalho = datetime.strptime(item['publishedAt'], '%Y-%m-%d %H:%M:%S')
            
            # Verifica se o trabalho está dentro do intervalo de datas
            if data_trabalho > data_ini and data_trabalho < data_fim:
                for query in queries:
                    palavra_chave = re.search(query, item['body'])
                    if palavra_chave:
                        if csv_check == 0:
                            # Adiciona dados para não CSV
                            alldata[item['id']] = [item['company']['name'], item['company']['url']]
                            not_found = 0
                        else:
                            # Adiciona dados para exportação em CSV
                            alldata[item['id']] = [
                                item.get('title', ''),
                                item['company'].get('name', ''),
                                item['company'].get('description', ''),
                                item.get('publishedAt', ''),
                                item.get('wage', ''),
                                item['company'].get('address', '')
                            ]
                            not_found = 0
            
            # Se a data de publicação for anterior à data inicial, já podemos parar a busca
            elif data_trabalho < data_ini:
                not_found = 0
                return alldata

    if not_found == 1:
        return False

    # Se o parâmetro 'csv' for passado nos argumentos, exporta para CSV
    if 'csv' in sys.argv and csv_check == 1:
        with open('skills_data.csv', mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Title', 'Company', 'Description', 'Published At', 'Salary', 'Location'])

            # Escreve os dados no CSV
            for job_id, job_data in alldata.items():
                writer.writerow(job_data)
        
        print(f"Informações exportadas para o arquivo skills_data.csv.")

    return alldata

In [45]:
a = top10_data()
print(a)


Exportação para CSV não foi solicitada.
{491896: ['Devoteam', '2024-11-05 22:25:31'], 492024: ['PTisp', '2024-11-05 22:25:02'], 491241: ['Noesis Portugal', '2024-11-05 22:24:49'], 490561: ['Dellent', '2024-11-05 22:24:28'], 491994: ['Randstad Portugal', '2024-11-05 22:24:12'], 492021: ['Sermicro', '2024-11-05 22:24:07'], 492027: ['Neotalent Conclusion', '2024-11-05 22:23:52'], 492008: ['Red IT', '2024-11-05 22:23:43'], 492020: ['Novo Atalho', '2024-11-05 22:23:38'], 491987: ['YellowIpe', '2024-11-05 22:23:32'], 491947: ['Aubay', '2024-11-05 22:23:29'], 491954: ['Landskill', '2024-11-05 22:23:25'], 491963: ['akapeople', '2024-11-05 22:23:21'], 491968: ['KCS IT', '2024-11-05 22:23:17'], 491978: ['Blip', '2024-11-05 22:23:14'], 491908: ['GRiT Solutions', '2024-11-05 22:23:10'], 491897: ['Devoteam', '2024-11-05 22:23:07'], 491181: ['Noesis Portugal', '2024-11-05 22:23:04'], 490537: ['Dellent', '2024-11-05 22:23:01'], 492028: ['Neotalent Conclusion', '2024-11-05 22:22:55'], 491993: ['Randst

In [46]:
result = search_data("Dellent", "Lisboa", 4)
print(result)  


Procurando trabalhos para Empresa: Dellent, Localidade: Lisboa, Número de Trabalhos: 4
Trabalho encontrado: PHP Developer na empresa Dellent na localidade Lisboa
Trabalho encontrado: SAP Cloud ALM Consultant na empresa Dellent na localidade Lisboa
Trabalho encontrado: Automation Tester na empresa Dellent na localidade Lisboa
Trabalho encontrado: Application Support Engineer na empresa Dellent na localidade Lisboa
4 trabalhos exportados para 'found_jobs.csv'.
[{'id': 490537, 'company': {'id': 2714, 'name': 'Dellent', 'logo': 'https://static.itjobs.pt/images/companies/b6/283/2714/logo.png', 'description': 'A Dellent é uma empresa de consultoria especializada em Sistemas de Informação e Telecomunicações. O nosso objetivo é conseguirmos ajudar os nossos consultores e candidatos a integrarem projetos que vão ao encontro do seus interesses, motivações e expectativas.', 'address': 'AVEIRO:\r\nCreative Science Park                \r\nVia do Conhecimento                  \r\n3830-352 Ílhavo\r\n

In [47]:
salary(12345)


Erro: Job not found.


In [48]:
skills = ["Python", "Django", "SQL"]
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
result = filter_jobs_by_skills_and_date(skills, start_date, end_date)

for job in result:
    print(job)

In [ ]:
############################################################# chamadas para csv ###################################################################################

In [ ]:
top10_data(export_csv=True)

Dados exportados para top10_jobs.csv.


{491896: ['Support Specialist',
  'Devoteam',
  'At Devoteam, we believe that technology with strong human values can actively drive change for the better. Discover how Tech for People unlocks the future, creating a positive impact on the people and the world around us. We are a global leading player in Digital Transformation for leading organizations across EMEA, with a revenue of €1B. We believe in transforming technology to create value for our clients, partners and employees in a world where technology is developed for people. We are proud of the culture we have built together. We are proud of our people at the service of technology. We are proud of our diverse environment. Because we are #TechforPeople. Join our multidisciplinary team of Cloud experts, Designers, Business consultants, Security experts, Engineers, Developers and other extraordinary talents, spread across more than 20 EMEA countries. Become one of our +10.000 tech and business leaders on cloud, data and cyber securi

In [ ]:
import sys
sys.argv = ['jobscli.py', 'csv']
search_data('Company Name', 'Location Name', 10)


Procurando trabalhos para Empresa: Company Name, Localidade: Location Name, Número de Trabalhos: 10
Nenhum trabalho encontrado para os critérios especificados.


In [ ]:
import sys
sys.argv = ['jobscli.py', 'salary', '12345', 'csv']
salary(12345)


Erro: Job not found.


In [ ]:
sys.argv = ['jobscli.py', 'skills', '["Python", "Django", "SQL"]', '2023-01-01', '2023-12-31', 'csv']

skills_data(
    ["Python", "Django", "SQL"], 
    datetime(2023, 1, 1), 
    datetime(2023, 12, 31), 
    csv_check=1
)

False